In [1]:
!unzip ./drive/MyDrive/single-person-pose-estimation/dataset/tfrecords.zip

Archive:  ./drive/MyDrive/single-person-pose-estimation/dataset/tfrecords.zip
   creating: dataset/tfrecords/
   creating: dataset/tfrecords/valid/
  inflating: dataset/tfrecords/valid/file_valid_03-208.tfrec  
  inflating: dataset/tfrecords/valid/file_valid_00-2048.tfrec  
  inflating: dataset/tfrecords/valid/file_valid_01-2048.tfrec  
  inflating: dataset/tfrecords/valid/file_valid_02-2048.tfrec  
   creating: dataset/tfrecords/train/
  inflating: dataset/tfrecords/train/file_train_23-2048.tfrec  
  inflating: dataset/tfrecords/train/file_train_54-2048.tfrec  
  inflating: dataset/tfrecords/train/file_train_09-2048.tfrec  
  inflating: dataset/tfrecords/train/file_train_33-2048.tfrec  
  inflating: dataset/tfrecords/train/file_train_28-2048.tfrec  
  inflating: dataset/tfrecords/train/file_train_29-2048.tfrec  
  inflating: dataset/tfrecords/train/file_train_38-2048.tfrec  
  inflating: dataset/tfrecords/train/file_train_55-2048.tfrec  
  inflating: dataset/tfrecords/train/file_train

In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
### For reading
def parse_tfrecord_fn(example):
    feature_description = {
        "ann_id": tf.io.FixedLenFeature([], tf.int64),
        "image_id": tf.io.FixedLenFeature([], tf.int64),
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_path": tf.io.FixedLenFeature([], tf.string),
        "width": tf.io.FixedLenFeature([], tf.int64),
        "height": tf.io.FixedLenFeature([], tf.int64),
        "keypoints/x": tf.io.VarLenFeature(tf.float32),
        "keypoints/y": tf.io.VarLenFeature(tf.float32),
        "keypoints/vis": tf.io.VarLenFeature(tf.int64),
        "keypoints/num": tf.io.FixedLenFeature([], tf.int64)
    }
    
    example = tf.io.parse_single_example(example, feature_description)
    example["image"] = tf.image.decode_image(example["image"], channels = 3, dtype = tf.float32, expand_animations =False)
    example["keypoints/x"] = tf.sparse.to_dense(example["keypoints/x"])
    example["keypoints/y"] = tf.sparse.to_dense(example["keypoints/y"])
    example["keypoints/vis"] = tf.sparse.to_dense(example["keypoints/vis"])
    return example
  ### example
def gen_examples_from_tfrecord(filepath, example_num):
  raw_dataset = tf.data.TFRecordDataset(filepath)
  parsed_dataset = raw_dataset.map(parse_tfrecord_fn)
  for example in parsed_dataset.take(example_num):
      for key in example.keys():
          if key != "image":
              print(f"{key}: {example[key]}")

      xcoords = example["keypoints/x"].numpy()
      ycoords = example["keypoints/y"].numpy()
      image = example["image"]
      h, w, c = image.shape
      print(f"Image shape: {image.shape}")
      plt.figure(figsize=(7, 7))
  
      plt.imshow(image)
      plt.scatter(xcoords  , ycoords  , marker = "o") # for heatmap size
      plt.show()  

In [5]:
gen_examples_from_tfrecord('dataset/tfrecords/valid/file_valid_00-2048.tfrec', 50)

Output hidden; open in https://colab.research.google.com to view.